In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib
import cufflinks as cf
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
import seaborn as sns
import random
import plotly.io as pio
import missingno as msno

cf.go_offline() # required to use plotly offline (no account required).
py.init_notebook_mode() # graphs charts inline (IPython).

## Data Cleaning

In [ ]:
# Run once to query data from 2016 to present
'''
url = "https://data.cityofnewyork.us/resource/erm2-nwe9.json"

# data_url='data.cityofnewyork.us'    # The Host Name for the API endpoint (the https:// part will be added automatically)
# data_set='5ymf-xnx3'    # The data set at the API endpoint (311 data in this case)
# app_token='XXXXX'   # The app token created in the prior steps
client = Socrata(data_url,app_token)      # Create the client to point to the API endpoint
# Set the timeout to 1000 seconds    
client.timeout = 1000
# Retrieve the first 2000 results returned as JSON object from the API
# The SoDaPy library converts this JSON object to a Python list of dictionaries
results = client.get(data_set, where="created_date>='2016-01-01T00:00:00.000'", limit=156_000)

# Convert the list of dictionaries to a Pandas data frame
df = pd.DataFrame.from_records(results)

df.to_csv('311_Service_Requests_from_2010_to_Present.csv', index=False)
'''

In [ ]:
call311 = pd.read_csv('311_Service_Requests_from_2010_to_Present.csv')

# convert cols to datetime format

call311['Created Date'] = pd.to_datetime(call311['Created Date'], format='%m/%d/%Y %I:%M:%S %p', errors='ignore')
call311['Closed Date'] = pd.to_datetime(call311['Closed Date'], format='%m/%d/%Y %I:%M:%S %p', errors='ignore')

# filter to 2016 to now
call311 = call311[call311['Created Date']>=dt.datetime(2016, 1, 1, 0, 0)]
call311.sort_values(by=['Created Date'], inplace=True)

call311.head()

In [ ]:
call311.info()

In [ ]:
call311.isnull().sum()/len(call311)*100

In [ ]:
# drop cols with predominantly null values and based on data dict understanding
call311 = call311.drop(["Descriptor", "Intersection Street 1", "Intersection Street 2", "Landmark","Facility Type",\
                        "Due Date", "BBL", "X Coordinate (State Plane)", "Y Coordinate (State Plane)", \
                        "Taxi Pick Up Location","Taxi Company Borough", "Vehicle Type","Bridge Highway Name", "Bridge Highway Direction", \
                        "Bridge Highway Segment", "Road Ramp"],axis=1)

In [ ]:
call311.isnull().sum() / len(call311)*100

In [ ]:
# visualize missing data (white spaces)
msno.matrix(call311)

In [ ]:
# duplicate original complaint type col
call311['Complaint Type Original'] = call311['Complaint Type']

# consolidate complaint types
call311['Complaint Type'] = call311['Complaint Type'].replace({'Noise - Residential':'Noise', 'Noise - Vehicle':'Noise',\
                                                               'Noise - Commercial':'Noise','Noise - Street/Sidewalk':'Noise',\
                                                               'Noise - Park':'Noise', 'Noise - House of Worship':'Noise', 'Noise - Helicopter':'Noise',\
                                                               'Collection Truck Noise':'Noise'})
# renamed unspecified noise type
call311['Complaint Type Original'] = call311['Complaint Type Original'].replace({'Noise':'Noise - Unspecified'})

In [ ]:
# featurize datetime column
call311['Day of Week'] = call311['Created Date'].dt.dayofweek
call311['Hour'] = call311['Created Date'].dt.hour
call311['Month'] = call311['Created Date'].dt.month
call311['Day'] = call311['Created Date'].dt.day

call311.head(5)

In [ ]:
call311['Incident Zip']

In [ ]:
'11210-000'.split('-')

In [ ]:
# clean up incident zipcode

(call311['Incident Zip'].str.split('-'))


In [ ]:
call311.shape

In [ ]:
# export data
call311.to_csv('311_filtered.csv', index=False)

## EDA
### Overall trends and statistics

In [ ]:
df = pd.read_csv('311_filtered.csv',dtype={'Incident Zipcode':'str'})

In [ ]:
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.lower()

In [ ]:
df.columns

In [ ]:
df['created_date'] = pd.to_datetime(df['created_date'])
df['created_timestamp'] = df['created_date']
df['created_date'] = df['created_date'].dt.date

temp = df.groupby('created_date').agg({'unique_key':'size'})
#                                         .rename({'unique_key':'cnt'}, inplace=True)

In [ ]:
fig = go.Figure([go.Scatter(x=temp.index, y=temp['unique_key'])])
fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(
    title={
        'text': "Daily 311 Calls",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [ ]:
df.complaint_type = df.complaint_type.str.title()
df.complaint_type_original = df.complaint_type_original.str.title()

df['complaint_type'].unique()

In [ ]:
# complaint types
df['complaint_type'].value_counts().iplot(kind='bar')
# note the long tail

In [ ]:
counts = df['complaint_type'].value_counts(dropna=False)

# counts for complaints above 90th percentile
valids = counts[counts>(np.percentile(counts, 95))].index

df[df['complaint_type'].isin(valids)]['complaint_type'].value_counts().iplot(kind='bar')

In [ ]:
# responding agency

def update_Office_SE(agency):

    """Takes in an agency name and converts to abbreviated form, if the agency is
     the Office of Special Enforcement."""

    if agency == 'MAYORâ\x80\x99S OFFICE OF SPECIAL ENFORCEMENT':
        return "OSE"
    else:
        return agency
    
df['agency'] = df['agency'].map(lambda agency: update_Office_SE(agency))

df['agency'].value_counts().iplot(kind='bar')

### Time series trends

In [ ]:
def hour_call_freq(agency):
    '''Takes in an agency name as a string. Returns a dataframe with agencies and the
    total number of calls related to the given agency that were made
    at each hour of the day.
    '''
    
    subset = df[df['agency']==agency]

    # init dict for storage
    call_freqs_by_hour = {}

    for hour in subset['hour']:

        if hour not in call_freqs_by_hour: 
            call_freqs_by_hour[hour] = 1

        else:
            call_freqs_by_hour[hour] += 1

    freq_df = pd.DataFrame.from_dict(call_freqs_by_hour, orient='index',
                                   columns=['{}'.format(agency)])

    return freq_df.sort_index()

In [ ]:
# ls = ['NYPD', 'HPD', '']

NYPD = hour_call_freq('NYPD')
HPD = hour_call_freq('HPD')
DOT = hour_call_freq('DOT')
DSNY = hour_call_freq('DSNY')
DEP = hour_call_freq('DEP')
DOB = hour_call_freq('DOB')
DPR = hour_call_freq('DPR')

# def check_idx_len(df):
#     if len(df) != 24:
#         print (df.columns+' contains missing index.')
#     else:
#         pass
    
# check_idx_len(NYPD)
# check_idx_len(DOF)
# assert len(DOT) == 24
# assert len(DSNY) == 24
# assert len(DPR) == 24
# assert len(HPD) == 24
# assert len(DEP) == 24
# assert len(DOB) == 24

In [ ]:
total = NYPD.NYPD+HPD.HPD+DOT.DOT+DSNY.DSNY+DEP.DEP+DOB.DOB+DPR.DPR

In [ ]:
font_dict = dict(size=20, color="#58508d", fontweight='bold')
title_font_dict = dict(size=27, color="#58508d", fontweight='bold')

plt.figure(figsize=(15, 15))
plt.plot(NYPD.index, NYPD.NYPD, label="NYPD", linewidth=5)
plt.plot(HPD.index, HPD.HPD, label="HPD", linewidth=5)
plt.plot(DOT.index, DOT.DOT, label="DOT", linewidth=5)
plt.plot(DSNY.index, DSNY.DSNY, label="DSNY", linewidth=5)

plt.plot(DEP.index, DEP.DEP, label="DEP", linewidth=5)
plt.plot(DOB.index, DOB.DOB, label="DOB", linewidth=5)
plt.plot(DPR.index, DPR.DPR, label="DPR", linewidth=5)
plt.plot(total.index, total, label="Total", linewidth=5)

plt.ylabel('Total Calls', fontdict=font_dict)
plt.xlabel('Hour', fontdict=font_dict)
plt.title('Total Calls by Hour and Agency', fontdict=title_font_dict)
plt.legend(title="Agency", title_fontsize=20, fontsize=18, 
           bbox_to_anchor=[1,1]) #position legend outside of plot
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlim(right=23) 
plt.xlim(left=0) 
plt.grid()

In [ ]:
def call_freq_df(complaint_type):
  
    """Takes in a complaint type as a string. Returns a dataframe with dates 
     and the total number of calls related to the given complaint that were made 
     on each date."""
    
    subset = df[df['complaint_type_original']==complaint_type]

    call_freqs_by_date = {}

    for date in pd.to_datetime(subset['created_date']):
#         print(type(date))
#         print(date.date())
        if date.date() not in call_freqs_by_date: 
            call_freqs_by_date[date.date()] = 1

        else:
            call_freqs_by_date[date.date()] += 1

    freq_df = pd.DataFrame.from_dict(call_freqs_by_date, orient='index', 
                                   columns=['{}'.format(complaint_type)])

    return freq_df

In [ ]:
def contain_substring(df, phrase):
    cols = []

    for i in df.complaint_type_original:
        if phrase in i.lower() and i not in cols:
            cols.append(i)
    return cols

In [ ]:
#create subsetted dataframes with complaints related to COVID-19
ls1 = contain_substring(df, 'complian')
ls2 = contain_substring(df, 'face cover')
name_ls = ls1+ls2

In [ ]:
reopen = call_freq_df('Noncompliance With Phased Reopening')
vacc = call_freq_df('Vaccine Mandate Non-Compliance')
vacc_priv = call_freq_df('Private School Vaccine Mandate Non-Compliance')
cover = call_freq_df('Face Covering Violation')

mgc = call_freq_df('Mass Gathering Complaint')

In [ ]:
#create and show area plot for COVID-related complaints
covid_fig = go.Figure()

plot_ls = [reopen, vacc, vacc_priv, cover]

for i in range(len(plot_ls)):
    
    covid_fig.add_trace(go.Scatter(
        x=plot_ls[i].index, y=plot_ls[i][name_ls[i]],
        hoverinfo='x+y',
        mode='lines',
        line=dict(width=0.5), name=name_ls[i],
        stackgroup='one' 
    ))

covid_fig.add_trace(go.Scatter(
    x=mgc.index, y=mgc['Mass Gathering Complaint'],
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='#000000'), name="Mass Gathering Complaint",
    stackgroup='two'
))
    

covid_fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.001
))


covid_fig.update_layout(title="COVID-19", height=700,
                  xaxis_title="Date",
                  yaxis_title="Total Calls", 

font=dict(family="silom",
          size=14, color="#58508d"))

covid_fig.show()

In [ ]:
#create subsetted dataframes with complaints related to noise
noise_r = call_freq_df('Noise - Residential')
noise_c = call_freq_df('Noise - Commercial')
noise_v = call_freq_df('Noise - Vehicle')
noise_s = call_freq_df('Noise - Street/Sidewalk')
noise = call_freq_df('Noise - Unspecified')

In [ ]:
#create and show area plot
noise_fig = go.Figure()

noise_fig.add_trace(go.Scatter(
    x=noise_r.index, y=noise_r['Noise - Residential'],
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='#003f5c'), name="Noise - Residential",
    stackgroup='one'
))
    
noise_fig.add_trace(go.Scatter(
    x=noise_s.index, y=noise_s['Noise - Street/Sidewalk'],
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='#58508d'), name='Noise - Street/Sidewalk',
    stackgroup='two' 
))

noise_fig.add_trace(go.Scatter(
    x=noise_v.index, y=noise_v['Noise - Vehicle'],
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='#bc5090'), name='Noise - Vehicle',
    stackgroup='three' 
))


noise_fig.add_trace(go.Scatter(
    x=noise_c.index, y=noise_c['Noise - Commercial'],
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='#ffa600'), name='Noise - Commercial',
    stackgroup='four' 
))

noise_fig.add_trace(go.Scatter(
    x=noise.index, y=noise['Noise - Unspecified'],
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='#003f5c'), name="Noise - Unspecified",
    stackgroup='one'
))

noise_fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99
))

noise_fig.update_layout(title="Noise", height=700,
                  xaxis_title="Date",
                  yaxis_title="Total Calls", 

font=dict(family="silom",
          size=14, color="#58508d"))

noise_fig.show()

In [ ]:
#create subsetted dataframes with complaints related to street and noise condition 
street = call_freq_df('Street Condition')
sidewalk = call_freq_df('Sidewalk Condition')

In [ ]:
#create and show area plot
condition_fig = go.Figure()
condition_fig.add_trace(go.Scatter(
    x=street.index, y=street["Street Condition"],
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='#22DDD3'), name="Street Condition",
    stackgroup='one'
    ))
    
condition_fig.add_trace(go.Scatter(
    x=sidewalk.index, y=sidewalk["Sidewalk Condition"],
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='#6722DD'), name="Sidewalk Condition",
    stackgroup='one' 


))

condition_fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99
))

condition_fig.update_layout(title="Sidewalk and Street Condition", height=700,
                  xaxis_title="Date",
                  yaxis_title="Total Calls", 

font=dict(family="silom",
          size=14, color="#58508d"))

condition_fig.show()

In [ ]:
#create subsetted dataframes with complaints related to trees
dt = call_freq_df('Damaged Tree')
ot = call_freq_df('Overgrown Tree/Branches')
dd = call_freq_df('Dead/Dying Tree')
nt = call_freq_df('New Tree Request')

In [ ]:
#create and show area plot (damaged trees)
damaged_tree_fig = go.Figure()
damaged_tree_fig.add_trace(go.Scatter(
    x=dt.index, y=dt["Damaged Tree"],
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='#228B22'), name="Damaged Trees",
    stackgroup='one'
    ))

damaged_tree_fig.update_layout(title="Damaged Trees", height=700,
                  xaxis_title="Date",
                  yaxis_title="Total Calls", 

font=dict(family="silom",
          size=14, color="#58508d"))

damaged_tree_fig.show()

## Heatmaps for call volume

In [ ]:
#Subset the data by month
June = df[df.month==6].sample(frac =.5).sort_values(by='day')
# July = df[df.month==7].sample(frac =.5).sort_values(by='day')
# August = df[df.month==8].sample(frac =.1).sort_values(by='day')
# September = df[df.month==9].sample(frac =.5).sort_values(by='day')
# October = df[df.month==10].sample(frac =.5).sort_values(by='day')
# November = df[df.month==11].sample(frac =.5).sort_values(by='day')

In [ ]:
June.head()

In [ ]:
def plot_calls(month_df, month):
  
    """Takes in a subsetted dataframe with the data for a given month, and the 
       name of the month as a string. Returns an animated mapbox density heatmap
       to show variation in call volume across time and space."""

    fig = px.density_mapbox(month_df, lat=month_df['latitude'], 
                            lon=month_df['longitude'], 
                            radius=2,
                            animation_frame=month_df["day"],
                            hover_name=None, 
                            hover_data=['complaint_type', 'community_board'],
                            width=550, height=550,
                            color_continuous_scale=px.colors.sequential.Inferno
                                                            )
    fig.update_layout(mapbox_style="carto-positron", mapbox_zoom=8.5, 
                  mapbox_center = {"lat": 40.6885, "lon": -73.93211},)
                      
    fig.layout.coloraxis.showscale = False    
                      
    fig.update_layout(title={
        'text': 'Call Volume in {}'.format(month),
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}),

    font=dict(family="silom",
              size=14, color="#58508d")

    fig.update_layout(transition = {'duration': 10})

    fig.show()

In [ ]:
plot_calls(June, 'June')